In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [5]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [7]:
#pip install category_encoders
#pip install --upgrade scipy

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import RobustScaler, Normalizer

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit, StratifiedKFold, RepeatedKFold

from sklearn.neighbors import KNeighborsRegressor

import sklearn

import pandas.plotting as pd_plt
import matplotlib.colors as plt_colors

from sklearn.metrics import make_scorer, recall_score, precision_score, f1_score, accuracy_score

from sklearn import svm

from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

import category_encoders as ce
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.linear_model import ElasticNet, Ridge, LinearRegression, Lasso, Lars
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from scipy import stats

### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [19]:
# Путь к тренировочному набору
path_train = 'train.csv'  # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [21]:
df = pd.read_csv(path_train)
X_test = pd.read_csv(path_test)

In [23]:
df['y']=np.where(df['y']>=15, df['y']/10,df['y'])
df['z']=np.where(df['z']>=15, df['z']/10,df['z'])

In [25]:
df.drop_duplicates(inplace=True)

In [27]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

In [42]:
X_train = df.drop(['price'], axis = 1)
y_train = df['price']

In [44]:
# Блок обучения модели

In [46]:
axis_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=0, strategy='mean')),
    ('polynom', PolynomialFeatures(2,include_bias=False)),
    ('scaler', StandardScaler())])

clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}
    }]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=clarity_map)),
    ('scaler', MinMaxScaler())
    ])

cut_map = [{
    'col':'cut',
    'mapping':{'Ideal':4, 'Premium':3, 'Very Good':2, 'Good':1, 'Fair':0}
    }]

cut_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=cut_map)),
    ('scaler', MinMaxScaler())
    ])

color_map = [{
    'col':'color',
    'mapping':{'I':6, 'E':5, 'D':4, 'H':3, 'F':2, 
               'G':1, 'J':0}
    }]

color_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=color_map)),
    ('scaler', MinMaxScaler())
    ])

carat_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(3,include_bias=False)),
    ('scaler', StandardScaler())])

In [48]:
axis_features = ['x','y','z']
num_features = ['depth','table']

In [50]:
CT = ColumnTransformer([
        ("axis_transformer", axis_transformer, axis_features),
        ("num", StandardScaler(), num_features),
        ("cut_transformer", cut_transformer, ['cut']),
        ("clarity_transformer", clarity_transformer, ['clarity']),
        ("carat_transformer", carat_transformer, ['carat']),
        ("color_transformer", color_transformer, ['color'])
        ])

display(CT)

ColumnTransformer(transformers=[('axis_transformer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(missing_values=0)),
                                                 ('polynom',
                                                  PolynomialFeatures(include_bias=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['x', 'y', 'z']),
                                ('num', StandardScaler(), ['depth', 'table']),
                                ('cut_transformer',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'cut',
                                                                           'data_type': dtype('O'...
                                                 ('scaler', MinMaxScaler())]),
                                 ['clarity']),
                                ('carat_transformer',
                                 Pipeline(steps=[('polynom',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['carat']),
                                ('color_transformer',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'color',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': I    6
E    5
D    4
H    3
F    2
G    1
J    0
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['color'])])

In [53]:
model =  Pipeline(steps=[
                                ('preprocessing', CT),
                                ('regressor', KNeighborsRegressor(n_neighbors = 5, p = 1, weights = 'distance'))
                            ])

model.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('axis_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(missing_values=0)),
                                                                  ('polynom',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['x', 'y', 'z']),
                                                 ('num', StandardScaler(),
                                                  ['depth', 'table']),
                                                 ('cut_transformer',
                                                  Pipeline(steps=[('ce',
                                                                   OrdinalEncoder(mapping=[{...
                                                  Pipeline(steps=[('polynom',
                                                                   PolynomialFeatures(degree=3,
                                                                                      include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['carat']),
                                                 ('color_transformer',
                                                  Pipeline(steps=[('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'color',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': I    6
E    5
D    4
H    3
F    2
G    1
J    0
dtype: int64}])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['color'])])),
                ('regressor', KNeighborsRegressor(p=1, weights='distance'))])

In [7]:
# Блок предсказания с использованием тестового набора

In [56]:
y_pred = model.predict(X_test)

In [58]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict = y_pred